# 本セクションの目次
1. データラングリングとは？
2. テーブル形式を含むExcelのラングリング
3. テーブル形式を含まないExcelのラングリング
4. PDFのラングリングを行ってみよう
5. ラングリングで気をつけること

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.apache.spark:spark-avro_2.12:3.2.1") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

# データラングリングとは？
データラングリングとは、データをこねくり回してデータをより使いやすくする作業のことを指します。

- 重複削除
- idから商品名を引っ張っってくる
- 使い物になる様に別テーブルとくっつける

データラングリングと呼ばれる対象は一般にはCSV、JSON、アクセスログもあるのですが、それ以外にもExcelのデータ、PDFのデータ
なども含まれています。

最終的な目標はテーブルの形式にするためにどの様にロジックを組むのか？というところに落ち着いてきます。

データラングリングというとかっこよく聞こえるかもしれないのですが、かなり地味な点と、エンジニアとしてラングリングを扱うには注意点がありますので  
その点について紹介をしていこうと思います。

データラングリングはPythonとPySparkを組み合わせながら進めていくことが多いです。

# テーブル形式を含むExcelのラングリング
Excelのラングリングは、Sparkで読み込みをすることができません。
そのためpandasを使ってExcelデータを読み込み、Sparkで処理をするということやってみたいと思います。

Excelのデータは比較的小さいので操作はPandasで行ってもいいのですが、今回はSparkで処理を行ってみたいと思います。

In [ ]:
import pandas as pd

df = pd.read_excel('./dataset/table_excel.xlsx')
print(df)



# テーブル形式を含まないExcelのラングリング

お次はテーブルっぽくないexcelのラングリングをしてみましょう。  
しかし心配は入りません。

Excelであればいつでも単純に処理をすることが可能です。


In [ ]:
import pandas as pd

df = pd.read_excel('./dataset/no_table_excel.xlsx')
print(df)



In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

#スキーマ設定をしていきましょう
struct = StructType([
    StructField("1", StringType(), False),
    StructField("2", StringType(), False),
    StructField("koumoku", StringType(), False),
    StructField("val", StringType(), False),
    StructField("kesssai", StringType(), False),
    StructField("name", StringType(), False),
])


exceldf=spark.createDataFrame(df,schema=struct)
exceldf.show()

In [ ]:
import pyspark.sql.functions as F
columns = exceldf.columns
for column in columns:
    exceldf = exceldf.withColumn(column,F.when(F.isnan(F.col(column)),None).otherwise(F.col(column)))

exceldf.show()

In [ ]:
exceldf=exceldf.dropDuplicates().select(exceldf.koumoku,exceldf.val,exceldf.kesssai,exceldf.name)
exceldf.show()

In [ ]:
exceldf=exceldf.dropna(how='all')
exceldf.show()

In [ ]:
exceldf.withColumn('koumoku',F.when(exceldf.koumoku.isNull(),exceldf.kesssai).otherwise(exceldf.koumoku)).show()
exceldf.withColumn('val',F.when(exceldf.val.isNull(),exceldf.name).otherwise(exceldf.val)).show()

result=exceldf.withColumn('koumoku',F.when(exceldf.koumoku.isNull(),exceldf.kesssai).otherwise(exceldf.koumoku))
result=result.withColumn('val',F.when(exceldf.val.isNull(),exceldf.name).otherwise(exceldf.val))

result.show()

In [ ]:
result=result.select(result.koumoku,result.val)
spark.createDataFrame(result.toPandas().set_index('koumoku').T).show()


result=spark.createDataFrame(result.toPandas().set_index('koumoku').T)

# これでやっと既存のテーブルなどと突合したりができる様になってきます

# PDFのラングリングを行ってみよう
PDFのラングリングは要注意です。
基本的にできることはできるのですが、出力したPDFの作り方によってはまともに読めないことがあります。

そのため、PDFのデータ解析をしたい！
という要望を受けたら、基本的には断りつつExcelに変更してもらうなどの対応をとる方が賢明です。

とはいえ、元のデータが残っておらずどうしてもやらなければならない時があるのでその時のために少しだけ方法を見てみましょう。
有効な方法は以下の２です。

- OCRでデータを読み取る(PyOCRなど)
- ガッツリデータを読み込む

今回はガッツリデータを読み込む方法で行ってみましょう。

In [ ]:
from re import split
from pdfminer.high_level import extract_text
import re
import os
from decimal import Decimal

text = extract_text(os.path.join("./dataset", "no_table_pdf.pdf"))

lines=text.split('\n')

#空行削除
lines = list(filter(None, lines))

for line in lines:
    print(line)

In [ ]:
# あとは表示されたアウトプットをもとに整形をしていくだけです

dict={}
dict[lines[0]]=lines[2]
dict[lines[1]]=lines[3]
dict[lines[4]]=lines[5]

print(dict)



In [ ]:
pd_dict=pd.DataFrame.from_dict(dict,orient='index')

print(pd_dict)
print("-------------")
print(pd_dict.transpose())

In [ ]:
pdf_spark=spark.createDataFrame(pd_dict.transpose()) 
pdf_spark.printSchema()
pdf_spark.show()

# データラングリングで気をつけること

ここまでみてどうだったでしょうか？
基本的にはできそうだけども。。

というところだったかと思います。

基本的に既に稼働しているアプリケーションは、データ分析を前提に作られていることはないのでこの様な作業が発生してしまいます。

そのため、必要であれば当然行うのですができる限りRDSなどの処理に落ち着ける様にできると良いかと思います。

特にPDFは沼にハマることが多いので、最低でもExcelなどに落ち着ける様に調整を行いましょう

In [ ]:
spark.stop()